# CSE4088 Machine Learning

### Göksel Tokur - 150116049

### Merve Ayer - 150119828

## Get Cryptocurrency Data From Binance API

## Merve Ayer - 150119828

In [1]:
from binance.client import Client
import os
import time
import pandas as pd
from datetime import timedelta, datetime
from dateutil import parser
import math
from tqdm import tqdm_notebook


from binance.websockets import BinanceSocketManager
from twisted.internet import reactor



In [2]:
# To run the script you need to get Binance API KEY.

API_KEY = 'YOU NEED TO WRITE API KEY HERE'
API_SECRET = 'YOU NEED TO WRITE API SECRET HERE'

binance_client = Client(API_KEY, API_SECRET)

In [3]:
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750


def dates_of_data(symbol, kline_size, data, source, date_from, date_to, date_to_isToday=True):
    if len(data) > 0:
        old = parser.parse(data["Timestamp"].iloc[-1])
    elif source == "binance": 
        old = datetime.strptime(date_from, '%d %b %Y')
    if source == "binance" and date_to_isToday == True: 
        new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    else:
        new = datetime.strptime(date_to, '%d %b %Y')
    print(old)
    print(new)

    return old, new


def get_binance_history(symbol, kline_size, date_from, date_to, date_to_isToday=True, save = False):
    name_string = 'data/%s-%s-%sto%sdata.csv'
    filename = (name_string % (symbol, kline_size, date_from, date_to)).replace(' ', '')
    if os.path.isfile(filename):
        data_df = pd.read_csv(filename)
    else:
        data_df = pd.DataFrame()
    
    oldest_point, newest_point = dates_of_data(symbol, kline_size, data_df, "binance", date_from, date_to, date_to_isToday)
    delta_min = (newest_point - oldest_point).total_seconds() / 60
    available_data = math.ceil(delta_min / binsizes[kline_size])
    
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'):
        print('Waiting to download %s data for %s.' % (kline_size, symbol))
    else:
        print('Waiting to download %d minutes of data for %s, %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close_time', 'Quote_av', 'Trades', 'Tb_base_av', 'Tb_quote_av', 'Ignore' ])
    data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='ms')
    
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else:
        data_df = data
    data_df.set_index('Timestamp', inplace=True)
    
    # if we want to save the data, save it.
    if save:
        data_df.to_csv(filename)
    print('Finished.')
    return data_df
#data = get_binance_history("BTCUSDT", "1m", save=True, date_from='1 Jun 2020', date_to='1 Dec 2020', date_to_isToday=False )


In [4]:
data = get_binance_history("BTCUSDT", "5m", save=True, date_from='1 Jun 2020', date_to='15 Jun 2021', date_to_isToday=False )

2020-06-01 00:00:00
2021-06-15 00:00:00
Waiting to download 545760 minutes of data for BTCUSDT, 109152 instances of 5m data.
Finished.
